In [1]:
# Step 1: Import library yang dibutuhkan
import requests
import re
import pandas as pd
from datetime import datetime

In [2]:
# Fungsi untuk mengonversi Unix timestamp ke format tanggal yang lebih mudah dibaca
def convert_timestamp(unix_time):
    return datetime.fromtimestamp(int(unix_time)).strftime('%Y-%m-%d %H:%M:%S')

In [3]:
# URL produk Shopee yang ingin di-scrape
# GANTI URL BERIKUT SESUAI DENGAN PRODUK YANG DIINGINKAN
url = 'https://shopee.co.id/MADEVINE-Hair-Oil-Obat-Rambut-Rontok-Dan-Penumbuh-Rambut-Pria-dan-Wanita-i.610215585.13462914547?sp_atk=51bcb430-de7a-4938-a185-d799879a5afc&xptdk=51bcb430-de7a-4938-a185-d799879a5afc'

In [4]:
# Gunakan regex untuk mengekstrak shop_id dan item_id dari URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r[1], r[2]
    print(f"Shop ID: {shop_id}, Item ID: {item_id}")
else:
    raise ValueError("Shop ID dan Item ID tidak dapat ditemukan di URL. Pastikan URL benar.")

Shop ID: 610215585, Item ID: 13462914547


In [5]:
# Template URL API Shopee untuk mendapatkan ulasan
ratings_url = 'https://shopee.co.id/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'

In [6]:
# Header untuk menyamarkan sebagai browser asli
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Referer": "https://shopee.co.id",
    "Accept-Language": "id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7"
}

In [7]:
import time

In [8]:
# Mulai dari offset 0 (halaman pertama)
offset = 0
d = {'username': [], 'rating': [], 'comment': [], 'date': []}

while True:
    response = requests.get(ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset), headers=headers)
    
    # Periksa apakah akses ditolak (403 Forbidden)
    if response.status_code == 403:
        print("Diblokir sementara, menunggu 10 detik...")
        time.sleep(10)
        continue  # Coba lagi setelah delay

    # Ubah response menjadi JSON
    data = response.json()

    # Pastikan data tersedia sebelum diproses
    if 'data' not in data or 'ratings' not in data['data']:
        print("Data tidak ditemukan atau sudah habis.")
        break

    for rating in data['data']['ratings']:
        review_date = convert_timestamp(rating.get('ctime', 0))  # Konversi timestamp
        d['username'].append(rating.get('author_username', 'N/A'))
        d['rating'].append(rating.get('rating_star', 0))
        d['comment'].append(rating.get('comment', 'Tidak ada komentar'))
        d['date'].append(review_date)

        print(f"User: {rating.get('author_username', 'N/A')}")
        print(f"Rating: {rating.get('rating_star', 0)}")
        print(f"Comment: {rating.get('comment', 'Tidak ada komentar')}")
        print(f"Date: {review_date}")
        print('-' * 80)

    # Hentikan jika jumlah ulasan kurang dari 20 (halaman terakhir)
    if len(data['data']['ratings']) < 20:
        break

    # Lanjut ke halaman berikutnya
    offset += 50
    time.sleep(2)  # Tambahkan delay untuk menghindari pemblokiran

print(f"Scraping selesai! Total ulasan yang diambil: {len(d['username'])}")

User: h*****o
Rating: 5
Comment: Tekstur:aromanya segar dan wangi
Cocok Untuk:rambut.
Performa:bagus

Produk original. Wangi dan segar. Baru pertama beli. Kalo berkhasiat nnti order lagi. Rambutku sebenarnya ga rontok. Pengen lebih tebal lagi aja. Siapa tau cocok.
Pengemasan dan pengiriman cepat. 


Tq
Date: 2023-07-06 13:51:48
--------------------------------------------------------------------------------
User: l*****1
Rating: 5
Comment: Pengalaman Penggunaan:Belum ad,First time
Wangi:Wanginya enak bgt rempah-rempah
Profil Kecantikan:Lumayan

Thankyou bgt paket saya baru saja sampai dgn selamat,packingannya jg ok.Baru pertama kli beli serum ini,klo lihat dr iklanhya,kyknya sih bgus,tp blum tau,krn belum ada bukti ke saya.Klo benar2 ada trbukti rambut saya dr kebotakan byk tumbuh,ya saya akan berlangganan disni.Thankyou...
Date: 2023-05-13 16:19:18
--------------------------------------------------------------------------------
User: d*****i
Rating: 5
Comment: Wangi:khas rempah
Manfaa

In [9]:
import pandas as pd

# Simpan hasil scraping ke dalam DataFrame
df = pd.DataFrame({
    'Username': d['username'],
    'Rating': d['rating'],
    'Comment': d['comment'],
    'Date': d['date']
})

# Simpan ke file Excel
excel_filename = 'shopee_reviews.xlsx'
df.to_excel(excel_filename, index=False, engine='openpyxl')

print(f"Data berhasil disimpan ke {excel_filename}")

Data berhasil disimpan ke shopee_reviews.xlsx
